In [1]:
#Reference for where i got most of the code from https://jrashford.com/2022/01/19/how-to-collect-data-from-reddit-introducing-praw/
import pandas as pd
import praw
from praw.models import MoreComments

In [2]:
reddit = praw.Reddit(client_id='ZeCMY9LXRnsY9M78sS8uGg', client_secret='V-MS7rfVXCJMWLvTfGNyjLLC85ycSg', user_agent='my-app', username='ForThethesis')


In [3]:
subreddit = reddit.subreddit('wallstreetbets')

In [4]:
name = subreddit.display_name
id_ = subreddit.id
public_description = subreddit.public_description
created_at = subreddit.created
print((name, id_, public_description, created_at))

('wallstreetbets', '2th52', 'Like 4chan found a Bloomberg Terminal.', 1328045167.0)


In [5]:
posts = []

In [6]:
for submission in subreddit.new(limit=1000):
    title = submission.title
    link = submission.url
    author = submission.author_fullname 
    score = submission.score
    created = submission.created    
    print((title, link, author, score, created))
    posts.append(title)

('Calls on Amazon, Costco, and Walmart? The lawyers are probably funded by them.', 'https://www.thestreet.com/retailers/only-amazon-costco-and-walmart-should-want-to-block-the-kroger-albertsons-deal?puc=yahoo&cm_ven=YAHOO&yptr=yahoo', 't2_ow1gjx4r', 0, 1675612941.0)
('Do you think eggs are expensive? Wait until you fertilize one', 'https://v.redd.it/su3wwthd6ega1', 't2_moabt381', 18, 1675612400.0)
('What makes option price to follow the Greeks. What enforces it?', 'https://www.reddit.com/r/wallstreetbets/comments/10ufc5f/what_makes_option_price_to_follow_the_greeks_what/', 't2_uzhc83ys', 3, 1675611468.0)
('Dagoth Ur joins WSB', 'https://v.redd.it/fzzxwxoqkfga1', 't2_1zpi4qpj', 19, 1675611329.0)
('Towels', 'https://i.redd.it/dl3spvb9jfga1.jpg', 't2_9rcjr565', 97, 1675610821.0)
('Few know what a pivot means', 'https://i.redd.it/umv4b5i4ifga1.jpg', 't2_5c2k1208', 16, 1675610439.0)
('finally down 100%', 'https://i.redd.it/d7lclvyxgfga1.jpg', 't2_a0lbxm31', 30, 1675610044.0)
('Does anyone k

In [7]:
df1 = pd.DataFrame({'All The Posts': posts})

In [8]:
df1

,All The Posts
0,"Calls on Amazon, Costco, and Walmart? The lawy..."
1,Do you think eggs are expensive? Wait until yo...
2,What makes option price to follow the Greeks. ...
3,Dagoth Ur joins WSB
4,Towels
...,...
895,Index Investors Watching us Lose Money Picking...
896,what are some good stocks to buy for people us...
897,WSB Big Casino Game Spot (with sound)
898,"MARKET IS DOOMED, NEW LIL YACHTY ALBUM JUST DR..."


In [9]:
body_list = [] 
author_list = []
created_list = []
score_list = []

In [10]:
submission.comments.replace_more(limit=0) # flatten tree
for tl_comment in submission.comments.list():
    body = tl_comment.body
    # author = tl_comment.author.name
    created = tl_comment.created
    score = tl_comment.score
    body_list.append(body)
    # author_list.append(author)
    created_list.append(created)
    score_list.append(score)

In [11]:
# df = pd.DataFrame({'col1': body_list, 'col2': author_list, 'col3': created_list, 'col4': score_list})

In [12]:
df = pd.DataFrame({'Comments': body_list})

In [13]:
df

,Comments
0,\n**User Report**| | | |\n:--|:--|:--|:--\n**T...
1,Ehang Holdings Ltd is a publicly traded compan...
2,Average price 14.10
3,Retire
4,You’re the best
5,Investing in Chinese stocks like throwing mone...
6,"Hey, if you haven't already, please reply to t..."
7,"I can predict any market , anyone want demo"
8,Never!


In [14]:
df.to_csv('The_List.csv', )

In [15]:
#This transformer model was taking from: https://huggingface.co/ahmedrachid/FinancialBERT-Sentiment-Analysis?text=Bids+or+offers+include+at+least+1%2C000+shares+and+the+value+of+the+shares+must+correspond+to+at+least+EUR+4%2C000.
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import pipeline

model = BertForSequenceClassification.from_pretrained("ahmedrachid/FinancialBERT-Sentiment-Analysis",num_labels=3)
tokenizer = BertTokenizer.from_pretrained("ahmedrachid/FinancialBERT-Sentiment-Analysis")

nlp = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

# sentences = ["Operating profit rose to EUR 13.1 mn from EUR 8.7 mn in the corresponding period in 2007 representing 7.7 % of net sales.",  
#              "Bids or offers include at least 1,000 shares and the value of the shares must correspond to at least EUR 4,000.", 
#              "Raute reported a loss per share of EUR 0.86 for the first half of 2009 , against EPS of EUR 0.74 in the corresponding period of 2008.", 
#              ]
results = nlp(The_List.csv)
print(results)

[{'label': 'positive', 'score': 0.9998133778572083},
 {'label': 'neutral', 'score': 0.9997822642326355},
 {'label': 'negative', 'score': 0.9877365231513977}]

c:\Users\maart\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


ImportError: 
BertForSequenceClassification requires the PyTorch library but it was not found in your environment. Checkout the instructions on the
installation page: https://pytorch.org/get-started/locally/ and follow the ones that match your environment.
Please note that you may need to restart your runtime after installation.
